# Python Fortgeschritten: Metaklassen
## Tag 4 - Notebook 20
***
In diesem Notebook wird behandelt:
- Metaklassen-Grundlagen
- type()
- __new__
- Anwendungsfälle
***


## 1 Metaklassen

Metaklassen sind die "Klassen von Klassen". Sie definieren, wie Klassen erstellt werden. Während eine normale Klasse definiert, wie Objekte (Instanzen) erstellt werden, definiert eine Metaklasse, wie Klassen selbst erstellt werden.

### Was sind Metaklassen?

In Python ist **alles ein Objekt**, auch Klassen. Klassen sind Instanzen von Metaklassen. Standardmäßig ist die Metaklasse `type`. Wenn Sie `class MyClass:` schreiben, wird Python intern `type('MyClass', bases, dict)` aufrufen, um die Klasse zu erstellen.

### Wann und warum verwenden?

Metaklassen sollten verwendet werden, wenn:
- **Framework-Entwicklung**: Bei der Entwicklung von Frameworks (z.B. Django, SQLAlchemy), die Klassen automatisch konfigurieren müssen. Metaklassen ermöglichen Kontrolle über den Prozess der Klassen-Erstellung.
- **Code-Generierung**: Wenn Code basierend auf Klassen-Definitionen zur Laufzeit generiert werden soll
- **Automatisierung**: Wenn automatisch Methoden, Attribute oder Validierungen zu Klassen hinzugefügt werden sollen
- **API-Konsistenz**: Wenn sichergestellt werden soll, dass alle Klassen bestimmte Methoden oder Attribute haben

### Vorteile

- **Mächtige Abstraktion**: Ermöglicht sehr flexible und mächtige Abstraktionen
- **DRY-Prinzip**: Kann wiederholten Code eliminieren, der sonst in jeder Klasse stehen müsste
- **Automatisierung**: Automatische Konfiguration von Klassen ohne manuellen Code
- **Framework-Unterstützung**: Ermöglicht elegante Framework-APIs

### Einschränkungen

- **Komplexität**: Metaklassen sind schwer zu verstehen und zu debuggen
- **Selten benötigt**: Die meisten Anwendungsfälle können mit einfacheren Alternativen gelöst werden
- **Performance**: Können die Klassen-Erstellung verlangsamen
- **Wartbarkeit**: Code mit Metaklassen ist schwerer zu warten und zu verstehen


In [ ]:
# type() kann Klassen dynamisch erstellen
MyClass = type('MyClass', (), {'x': 10})
obj = MyClass()
print(obj.x)


## 2 type() und Klassen-Erstellung

Die Funktion `type()` kann auf zwei Arten verwendet werden:
1. **Als Funktion**: `type(obj)` gibt den Typ eines Objekts zurück
2. **Als Metaklasse**: `type(name, bases, dict)` erstellt eine neue Klasse

### Wie werden Klassen erstellt?

Wenn Sie `class MyClass:` schreiben, führt Python intern folgendes aus:

In [ ]:
# class MyClass: pass
# wird zu:
MyClass = type('MyClass', (), {})

# Beispiel 1: Klasse mit Attributen und Methoden
# Entspricht: 
# class Person: 
#   name = "Max"; 
# def greet(self): 
#   return f"Hallo, ich bin {self.name}"
def greet(self):
    return f"Hallo, ich bin {self.name}"

Person = type('Person', (), {'name': 'Max', 'greet': greet})
person = Person()
print(person.greet())  # Ausgabe: Hallo, ich bin Max

# Beispiel 2: Klasse mit Vererbung
# Entspricht: 
# class Animal: 
#   pass; 
# class Dog(Animal): 
#   species = "Canis"
Animal = type('Animal', (), {})
Dog = type('Dog', (Animal,), {'species': 'Canis'})
dog = Dog()
print(f"Dog ist eine Instanz von Animal: {isinstance(dog, Animal)}")  # True
print(f"Spezies: {dog.species}")  # Ausgabe: Canis

Die Parameter von `type()` sind:
- **name**: Der Name der Klasse (String)
- **bases**: Ein Tupel von Basisklassen
- **dict**: Ein Dictionary mit Attributen und Methoden der Klasse

### Das Metaklass-Protokoll

Eine Metaklasse muss die Methode `__new__()` implementieren, die die Klasse erstellt:
- `__new__(cls, name, bases, dct)`: Wird aufgerufen, um eine neue Klasse zu erstellen
- `cls`: Die Metaklasse selbst
- `name`: Der Name der zu erstellenden Klasse
- `bases`: Die Basisklassen
- `dct`: Das Dictionary mit Klassen-Attributen und -Methoden

## 3 Eigene Metaklasse

Wir können eigene Metaklassen erstellen, die Klassen-Verhalten kontrollieren.

In [ ]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        dct['created_by'] = 'Meta'
        return super().__new__(cls, name, bases, dct)

class MyClass(metaclass=Meta):
    pass

print(MyClass.created_by)


In [ ]:
# Metaklasse, die nur Klassen erlaubt, die Setter und Getter für alle Attribute haben
class PropertyRequiredMeta(type):
    """
    Metaklasse, die sicherstellt, dass alle Attribute (außer privaten und speziellen)
    als Properties mit Getter und Setter definiert sind.
    """
    def __new__(cls, name, bases, dct):
        # Sammle alle Properties (Getter) aus der Klasse
        properties = {}
        for key, value in dct.items():
            # Prüfe ob es ein Property ist (hat fget und fset)
            if isinstance(value, property):
                properties[key] = value
        
        # Prüfe alle Attribute, die keine Properties sind
        # Ignoriere private Attribute (beginnen mit _) und spezielle Attribute (__module__, __qualname__, etc.)
        for key in dct.keys():
            # NUR öffentliche Attribute prüfen (nicht mit _ beginnend)
            if not key.startswith('_'):
                attr = dct[key]
                # Ignoriere Methoden und Klassen-Variablen
                if not callable(attr) and not isinstance(attr, (type, classmethod, staticmethod)):
                    # Es ist ein öffentliches Attribut - muss ein Property sein
                    prop = properties.get(key)
                    if prop is None:
                        raise TypeError(
                            f"Klasse '{name}': Attribut '{key}' muss als Property "
                            f"mit Getter und Setter definiert sein. "
                            f"Verwende @property und @{key}.setter"
                        )
                    # Prüfe ob es einen Setter hat
                    if prop.fset is None:
                        raise TypeError(
                            f"Klasse '{name}': Property '{key}' hat keinen Setter. "
                            f"Verwende @{key}.setter"
                        )
        
        return super().__new__(cls, name, bases, dct)


# Beispiel 1: Korrekte Verwendung - alle Attribute haben Properties
class Person(metaclass=PropertyRequiredMeta):
    def __init__(self, name, age):
        self._name = name
        self._age = age
    
    @property
    def name(self):
        """Getter für name"""
        return self._name
    
    @name.setter
    def name(self, value):
        """Setter für name"""
        if not isinstance(value, str):
            raise TypeError("Name muss ein String sein")
        self._name = value
    
    @property
    def age(self):
        """Getter für age"""
        return self._age
    
    @age.setter
    def age(self, value):
        """Setter für age"""
        if not isinstance(value, int) or value < 0:
            raise ValueError("Alter muss eine positive Ganzzahl sein")
        self._age = value


# Verwendung
person = Person("Max", 25)
print(f"{person.name} ist {person.age} Jahre alt")  # Max ist 25 Jahre alt
person.name = "Anna"
person.age = 30
print(f"{person.name} ist {person.age} Jahre alt")  # Anna ist 30 Jahre alt


# Beispiel 2: Fehler - Klassen-Attribut ohne Property
try:
    class InvalidPerson(metaclass=PropertyRequiredMeta):
        # Dies ist ein öffentliches Klassen-Attribut ohne Property -> Fehler!
        name = "Default Name"
        
        # Lösung wäre: Property mit Getter und Setter definieren
        # @property
        # def name(self):
        #     return self._name
        # 
        # @name.setter
        # def name(self, value):
        #     self._name = value
        
        def __init__(self, name):
            self._name = name
except TypeError as e:
    print(f"Fehler erwartet: {e}")

# Hinweis: Instanz-Attribute (self.name in __init__) werden NICHT geprüft,
# da die Metaklasse nur zur Klassen-Definitionszeit ausgeführt wird,
# nicht zur Instanziierungszeit!

### Typische Anwendungsfälle:
- Singleton-Pattern - Nur eine Instanz einer Klasse erlauben
- Plugin-/Class-Registry - Automatisches Registrieren von Klassen
- ORM (Object-Relational Mapping) - Django Models, SQLAlchemy
- API-Validierung - Sicherstellen, dass Klassen bestimmte Methoden haben
- Automatisches Logging/Debugging - Methoden automatisch instrumentieren

## 4 Best Practices

### Best Practices

1. **Alternativen zuerst prüfen**: In 90% der Fälle sind Decorators, Klassen-Fabriken oder `__init_subclass__` einfachere Lösungen.

2. **Nur wenn wirklich nötig**: Metaklassen sollten nur verwendet werden, wenn keine einfachere Alternative existiert.

3. **Klare Dokumentation**: Metaklassen sind komplex - ausführliche Dokumentation ist essentiell.

4. **Einfache Metaklassen**: Halten Sie Metaklassen so einfach wie möglich. Komplexität macht Code unwartbar.

5. **Performance beachten**: Metaklassen werden bei jeder Klassen-Definition ausgeführt. Vermeiden Sie teure Operationen.

### Alternativen zu Metaklassen

**Decorators**: Für Funktionen, die auf Klassen angewendet werden sollen

In [ ]:
@register_plugin
class MyPlugin:
    pass

**__init_subclass__**: Für Verhalten, das bei der Klassen-Definition ausgeführt wird (Python 3.6+)

In [ ]:
class Base:
    def __init_subclass__(cls, **kwargs):
        # Wird bei jeder Subklassen-Definition aufgerufen
        pass

**Klassen-Fabriken**: Funktionen, die Klassen zurückgeben

In [ ]:
def create_class(name, attributes):
    return type(name, (), attributes)

### Wann NICHT verwenden?

- **Einfache Anwendungsfälle**: Wenn Decorators oder `__init_subclass__` ausreichen
- **Einmalige Anwendung**: Wenn das Verhalten nur einmal benötigt wird
- **Performance-kritisch**: Wenn Klassen sehr häufig erstellt werden

### Häufige Fehler

- **Zu komplex**: Metaklassen, die zu viel tun und schwer zu verstehen sind
- **Fehlende Dokumentation**: Metaklassen ohne Erklärung sind für andere unverständlich
- **Vergessene Basisklassen**: Bei Vererbung müssen Metaklassen korrekt weitergegeben werden

#### 5.1 Aufgaben:

> (a) Erstelle eine Metaklasse, die automatisch ein `__str__` für alle Klassen hinzufügt. <br>



In [ ]:
# Deine Lösung:



> (b) Verwende `type()` um eine Klasse dynamisch zu erstellen. Teste mit einer Instanz

In [ ]:
# Deine Lösung:



#### Lösung:


In [ ]:
# Musterlösung (a)
class AutoStrMeta(type):
    def __new__(cls, name, bases, dct):
        if '__str__' not in dct:
            def __str__(self):
                return f"{name} instance"
            dct['__str__'] = __str__
        return super().__new__(cls, name, bases, dct)

class Test(metaclass=AutoStrMeta):
    pass

t = Test()
print(str(t))

# Musterlösung (b)
DynamicClass = type('DynamicClass', (), {'value': 42})
obj = DynamicClass()
print(obj.value)
